<a href="https://colab.research.google.com/github/tellme0823/100-nlp-papers/blob/master/Text_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets


  Using cached datasets-2.13.1-py3-none-any.whl (486 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00


In [7]:
from datasets import load_dataset
dataset = load_dataset('ms_marco', 'v1.1')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

Dataset ms_marco downloaded and prepared to /root/.cache/huggingface/datasets/ms_marco/v1.1/1.1.0/b6a62715fa5219aea5275dd3556601004cd63945cb63e36e022f77bb3cbbca84. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
subset = dataset['test']
queries_infos = []
queries = []
corpus = []

for sample in subset :
  query_type = sample ['query_type']
  if query_type != 'entity':
    continue
  query_id = sample ['query_id']
  query_str = sample ['query']
  passages_dict = sample ['passages']
  is_selected_lst = passages_dict ['is_selected']
  passage_text_lst = passages_dict ['passage_text']
  query_info = {
      'query_id ': query_id ,
      'query': query_str ,
      'relevant_docs': []
      }
  current_len_corpus = len (corpus)
  for idx in range (len ( is_selected_lst ) ) :
    if is_selected_lst [idx] == 1:
      doc_idx = current_len_corpus + idx
      query_info ['relevant_docs'].append(doc_idx)

    if query_info ['relevant_docs'] == []:
      continue

  queries.append(query_str)
  queries_infos.append (query_info)
  corpus += passage_text_lst


In [15]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download ('stopwords')
english_stopwords = stopwords.words ('english')
remove_chars = string.punctuation
stemmer = PorterStemmer()

def text_normalize (text):
  text = text.lower()
  for char in remove_chars:
    text = text.replace(char, '')
    text = ''.join([word for word in tokenize (text) if word not in english_stopwords])
    text = ''.join([stemmer.stem (word) for word in tokenize(text)])

  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
def create_dictionary(corpus ):
  dictionary = []
  for doc in corpus:
    normalized_doc = text_normalize(doc)
    tokens = tokenize(normalized_doc)
    for token in tokens:
      if token not in dictionary:
        dictionary.append(token)

  return dictionary

In [18]:
def vectorize (text, dictionary):
  word_count_dict = {word: 0 for word in dictionary}
  tokens = tokenize(text)
  for token in tokens:
    try:
      word_count_dict[token] += 1
    except:
      pass
  vector = list(word_count_dict.values())

  return vector

In [19]:
def create_doc_term_matrix(corpus, dictionary):
  doc_term_matrix = {}
  for idx, doc in enumerate(corpus):
    normalized_doc = text_normalize(doc )
    vector = vectorize(normalized_doc, dictionary)
    doc_term_matrix[(doc, idx)] = vector

  return doc_term_matrix

In [20]:
from scipy import spatial

def similarity(a, b):
  return 1 - spatial.distance.cosine(a, b)

In [ ]:
def ranking(query, dictionary, doc_term_matrix):
  normalized_query = text_normalize (query)
  query_vec = vectorize(normalized_query, dictionary)
  scores = []
  for doc_info, doc_vec in doc_term_matrix.items() :
    sim = similarity(query_vec, doc_vec)
    scores.append((sim, doc_info))
    scores.sort(reverse = True)

  return scores